# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [53]:
# your code here
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Step 1: Read the CSV file into a DataFrame
print("Step 1: Read the CSV file into a DataFrame")
df = pd.read_csv('C:/Users/Ignacio/IronHackCodes/gitHStuff/Labs/lab-neural-networks-master/your-code/tic-tac-toe.csv')

# Step 2: Inspect the dataset
print("Step 2: Inspect the dataset")
print(df.info())
print(df.describe())
print(df.head())

# Step 3: Convert categorical values to numeric
print("Step 3: Convert categorical values to numeric")
label_encoder = LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

print(df.head())

# Step 4: Separate inputs and output
print("Step 4: Separate inputs and output")
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

print(X.head())
print(y.head())

# Step 5: Normalize the input data
print("Step 5: Normalize the input data")
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

print(X_normalized[:5])  # Print the first 5 rows of the normalized data

Step 1: Read the CSV file into a DataFrame
Step 2: Inspect the dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None
         TL   TM   TR   ML   MM   MR   BL   BM   BR class
count   958  958  958  958  958  958  958  958  958   958
unique    3    3    3    3    3    3    3    3    3     2
top       x    x    x    x    x    x    x    x    x  True
freq    418  378  418  378  458  378  418  378  418   626
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x 

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [5]:
!pip install tensorflow

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/390.3 MB 8.4 MB/s eta 0:00:47
   - -------------------------------------- 12.8/390.3 MB 33.6 MB/s eta 0:00:12
   -- ------------------------------------- 26.0/390.3 MB 44.4 MB/s eta 0:00:09
   ---- ----------------------------------- 39.3/390.3 MB 49.0 MB/s eta 0:00:08
   ----- ---------------------------------- 52.4/390.3 MB 51.3 MB/s eta 0:00:07
   ------ ----------------

In [11]:
!pip install keras

In [55]:
# your code here
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Step 1: Split the training and test data
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Step 2: Create a Sequential model
print("Step 2: Create a Sequential model")
model = Sequential()

# Step 3: Add layers to the model
print("Step 3: Add layers to the model")
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

# Step 4: Compile the model using adam
print("Step 4: Compile the model using adam")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 5: Fit the training data
print("Step 5: Fit the training data")
history = model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2)

# Step 6: Evaluate the model with the test data
print("Step 6: Evaluate the model with the test data")
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Loss: {test_loss}')

# Step 7: Save the model
print("Step 7: Save the model")
model.save('tic-tac-toe.keras')
print('Model saved as tic-tac-toe.keras')

Step 2: Create a Sequential model
Step 3: Add layers to the model
Step 4: Compile the model using adam
Step 5: Fit the training data
Epoch 1/50


C:\Users\Ignacio\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6845 - loss: 0.6374 - val_accuracy: 0.6818 - val_loss: 0.6301
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6450 - loss: 0.6346 - val_accuracy: 0.6818 - val_loss: 0.6106
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6154 - loss: 0.6454 - val_accuracy: 0.6818 - val_loss: 0.6057
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6176 - loss: 0.6452 - val_accuracy: 0.6818 - val_loss: 0.5939
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6286 - loss: 0.6080 - val_accuracy: 0.6818 - val_loss: 0.5902
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6559 - loss: 0.5996 - val_accuracy: 0.6883 - val_loss: 0.5784
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6493 - loss: 0.5718 - val_accuracy: 0.7273 - val_loss: 0.5736
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6737 - loss: 0.5596 - val_accuracy: 0.7597 - val_loss: 0.5655
Epo

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [57]:
# your code here
import numpy as np
from tensorflow.keras.models import load_model

# Step 1: Load the saved model
print("Step 1: Load the saved model")
loaded_model = load_model('tic-tac-toe.keras')
print("Model loaded successfully.")

# Step 2: Select a few random rows from the test dataset
print("Step 2: Select random rows from the test dataset")
num_samples = 5  # Number of rows to test
indices = np.random.choice(len(X_test), num_samples, replace=False)
X_sample = X_test[indices]
y_sample_actual = y_test.iloc[indices]

# Step 3: Make predictions
print("Step 3: Make predictions")
predictions = loaded_model.predict(X_sample)

# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Step 4: Compare predictions with actual labels
print("Step 4: Compare predictions with actual labels")
for i in range(num_samples):
    print(f"Sample {i+1}:")
    print(f"  Predicted class: {predicted_classes[i]}")
    print(f"  Actual class:    {y_sample_actual.iloc[i]}")

Step 1: Load the saved model
Model loaded successfully.
Step 2: Select random rows from the test dataset
Step 3: Make predictions
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Step 4: Compare predictions with actual labels
Sample 1:
  Predicted class: 1
  Actual class:    1
Sample 2:
  Predicted class: 0
  Actual class:    0
Sample 3:
  Predicted class: 1
  Actual class:    1
Sample 4:
  Predicted class: 1
  Actual class:    1
Sample 5:
  Predicted class: 0
  Actual class:    0


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [65]:
# your code here
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Step 1: Rebuild the model with more layers
print("Step 1: Rebuild the model with more layers")
improved_model = Sequential()

# Add layers
improved_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
improved_model.add(Dropout(0.3))  # Increase dropout to prevent overfitting

improved_model.add(Dense(64, activation='relu'))
improved_model.add(Dropout(0.3))

improved_model.add(Dense(32, activation='relu'))
improved_model.add(Dropout(0.3))

improved_model.add(Dense(16, activation='relu'))
improved_model.add(Dropout(0.3))

improved_model.add(Dense(2, activation='softmax'))

# Step 2: Adjust the learning rate
print("Step 2: Adjust the learning rate")
custom_optimizer = Adam(learning_rate=0.01)  # Experimenting with 0.0005, 0.001, and *0.01
#0.001 = 0.9531/0.1197
#0.0005 = 0.9323/0.1905
#0.01 = 1.0000/0.0073 also 0.9948/0.0122 also 0.9843/0.0407

improved_model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 3: Add EarlyStopping callback and increase epochs
print("Step 3: Add EarlyStopping and fit the model")
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the improved model
history = improved_model.fit(
    X_train, y_train,
    epochs=100,  # Increased number of epochs
    batch_size=16,  # testing batch sizes: 8, *16, or 32
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Step 4: Evaluate the improved model
print("Step 4: Evaluate the improved model")
test_loss, test_accuracy = improved_model.evaluate(X_test, y_test)
print(f"Improved Test Accuracy: {test_accuracy}")
print(f"Improved Test Loss: {test_loss}")

Step 1: Rebuild the model with more layers
Step 2: Adjust the learning rate
Step 3: Add EarlyStopping and fit the model
Epoch 1/100


C:\Users\Ignacio\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6660 - loss: 0.6796 - val_accuracy: 0.6883 - val_loss: 0.6166
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6347 - loss: 0.6555 - val_accuracy: 0.6818 - val_loss: 0.6011
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6623 - loss: 0.6130 - val_accuracy: 0.7532 - val_loss: 0.5733
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7219 - loss: 0.5701 - val_accuracy: 0.7532 - val_loss: 0.5575
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7603 - loss: 0.5712 - val_accuracy: 0.7792 - val_loss: 0.4915
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7646 - loss: 0.5448 - val_accuracy: 0.8052 - val_loss: 0.5029
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7500 - loss: 0.4953 - val_accuracy: 0.7987 - val_loss: 0.5048
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8072 - loss: 0.4505 - val_accuracy: 0.8182 - val_loss: 0.4

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
#1) Adding more layers
#2) Increase dropout to prevent overfitting
#3) Adjust the learning_rate of the adam optimizer
#4) Increased number of epochs
#5) Adjusting the batch sizes
#6) Impliment EarlyStopping in the fit so that it stops training when the model, which stops "improving", saving time and preventing overfitting.